# Example 2: Writing Data with Compression Options

This example demonstrates various ways to write DataContainer objects to HDF5 files.

**Note:** All generated files will be saved to the `example_saving_data/` folder

In [22]:
from pythermondt.readers import LocalReader
from pythermondt.writers import LocalWriter, S3Writer

Now reader and writer objects can be created. The LocalWriter will save files to the `example_saving_data` folder.

**Note** In this example the S3Writer class is used to upload the data to an AWS S3 bucket. For this to work you need setup your AWS credentials. This can be done in several ways (see [here](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html#guide-configuration))

In [23]:
reader = LocalReader(pattern="./example_data", recursive=True)
writer = LocalWriter(destination_folder="./example_saving_data")
uploader = S3Writer(bucket="ffg-bp", prefix="example2_writing_data")

Now we can load data using the reader object, manipulate it, and write it back to a file using the writer object. Writers always write their data to an HDF5 File. The data can be read back using a reader object.

The files can also be viewed using: [HDF5 Viewer](https://myhdf5.hdfgroup.org/)

## Compression Options

PyThermoNDT supports different compression methods for HDF5 files:
- `"lzf"`: Fast compression (default) - good balance of speed and compression ratio
- `"gzip"`: Better compression but slower - use for smaller files when speed is less critical
- `"none"`: No compression - fastest read/write operations but largest file sizes

You can specify compression when writing files using the `compression` and `compression_opts` parameters.

In [24]:
for i, container in enumerate(reader):
    container.add_attributes("/MetaData", author="Max Mustermann", version=12.6)

    # Write with default compression (lzf) - good balance of speed and size
    writer.write(container, f"example-local-{i}.hdf5")

    # Write with gzip compression for smaller files
    writer.write(container, f"example-local-{i}-small.hdf5", compression="gzip", compression_opts=6)

## Writing with Different Writer Types

The same compression options work with any writer object. Here's an example using the S3Writer to upload data to AWS S3 with the highest compression settings:

In [25]:
# Example with S3Writer - upload to AWS S3 bucket
for i, container in enumerate(reader):
    container.add_attributes("/MetaData", author="S3 Upload Example", version=2.0)

    # Upload with default compression
    uploader.write(container, f"example-s3-{i}.hdf5", compression="gzip", compression_opts=9)

print("Files uploaded to S3 bucket")

Files uploaded to S3 bucket


## Direct HDF5 Saving

You can also save DataContainer objects directly to HDF5 files without using writer objects. This is useful for simple cases where you don't need the additional features of writers (like automatic file extension handling or backend abstraction).

The `save_to_hdf5()` method supports the same compression options as the writers.

In [26]:
# Direct saving to HDF5 files
container = reader[0]
container.add_attributes("/MetaData", author="Direct Save Example", version=1.0)

# Save with no compression for maximum speed
container.save_to_hdf5(f"./example_saving_data/direct-save{i}.hdf5", compression="none")

print("File saved directly using container.save_to_hdf5() method")

File saved directly using container.save_to_hdf5() method
